<div style="text-align: center;">
    <h1>Tutorial for using MLflow on Puhti</h1>
</div>


<div style="text-align: center;">
    <img src="./pics/CSC_2012_LOGO.png" alt="CSC_2012_LOGO.png" width="400" height="300">
</div>


This tutorial will guide you through using MLflow in the Puhti computing environment, offering a streamlined and centralized approach to tracking machine learning experiments. It’s tailored for machine learning practitioners who seek an efficient way to manage and monitor their experiments.

While prior experience with MLflow isn’t necessary, a basic understanding of supercomputing is recommended. We’ll explore the core components of MLflow and demonstrate their application through practical examples. You can follow along with the provided sample code or incorporate your own code into the tutorial.

The sample code creates and trains two neural network models for classifying MNIST digits. **Model 1**, `ReLU_2Layer_Adagrad`, uses ReLU activation with the Adagrad optimizer. **Model 2**, `ReLU_2Layer_Adam`, has the same architecture but uses the Adam optimizer. Their performance is compared using MLflow.

### What is MLflow?

**MLflow** in an open-source tool for managing machine learning models throughout their life cycle. It has four key components that can be widely utilized, from experimenting to deploying models:

- [**Tracking Server**](https://mlflow.org/docs/latest/tracking.html) is the core component used for tracking experiments. Results can be viewed and compared through an informative user interface or API.
	
- [**Models**](https://mlflow.org/docs/latest/models.html) is for packaging the models in a unified format, making it easy to move and share them.

- [**Model Registry**](https://mlflow.org/docs/latest/model-registry.html) provides tools for registering and versioning models. The registry can also be managed through the UI.

- [**Projects**](https://mlflow.org/docs/latest/projects.html) is for packaging entire ML project code, enabling easy sharing and reproducibility.

By organizing your work into ***experiments*** and ***runs***, MLflow ensures that you can systematically track progress, compare results, and refine your models effectively.

For more info on components visit [MLflow documentation](https://mlflow.org/docs/latest/introduction/index.html#what-is-mlflow).

In [ ]:
# Let's make sure we have the latest version of MLflow (>2.15.1) in use. If not, run pip install below:
!mlflow --version
#!pip install --upgrade mlflow

In [ ]:
# With sample code, we use Grdaio Interface to demo deployment and inference:
!pip install gradio

In [ ]:
import pandas as pd
import requests 
import os
import numpy as np

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical

import mlflow

import gradio as gr
from skimage.transform import resize

---

## MLflow Tracking Server & Models

### About Storing Artifacts and Metadata

In the example code, we use local storage for both metadata and artifacts. Additionally, MLflow supports various options for selecting the backend and artifact storage.

- **Backend Store**: This is where ***metadata*** is stored, including information about each run. By default, MLflow saves this metadata locally in the `mlruns` directory. However, it can be configured to use an external database, such as PostgreSQL or MySQL. You can read more about backend stores here: [MLflow Backend Stores](https://mlflow.org/docs/latest/tracking/backend-stores.html#backend-stores).

- **Artifact Store**: This is where the ***artifacts*** generated during runs—such as model weights, model files, and data files—are stored. The **Models component** is used to package these model files in a standardized format. Similar to the backend store, MLflow defaults to using the local `mlruns` directory for artifact storage, but it can be set to use external storage, such as S3 object storage like Allas. For more information, refer to [MLflow Artifact Stores](https://mlflow.org/docs/latest/tracking/artifacts-stores.html) and the [CSC documentation on Allas](https://docs.csc.fi/computing/allas/) and [using Allas with Python and Boto3](https://docs.csc.fi/data/Allas/using_allas/python_boto3/).

In the example code below, local directory is used for the backend and artifact storage.

### Configuring MLflow

First, we activate the **Tracking Server** component:

Start by setting the tracking URI, which defines the path where MLflow will create the `mlruns` directory to store metadata and artifacts generated during runs. 
If no path is provided, MLflow will create the `mlruns` directory in the location where the code is executed.

Next, set up an experiment under which the upcoming training runs will be logged. If the experiment does not already exist, it will be created.

In [ ]:
# Set tracking URI
project_id = "project_2001234" # Insert CSC your project_id here
mlruns_uri = f"scratch/{project_id}/path/to/mlruns" # URI for desired storage, you can decide the location
mlflow.set_tracking_uri(mlruns_uri) # https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow-tracing-apis

# Set experiment
experiment_name = "MLflow tutorial"
experiment = mlflow.set_experiment(experiment_name) # https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow-tracing-apis
print(f"Artifacts are saved stored here: {experiment.artifact_location}")

### Experimenting with MLflow

Next, we will run training sessions using MLflow. We'll utilize the autolog function, which automatically logs all the data generated during the run. 
By default, the model will be logged as an artifact, making it easy to access later on and enabling automatic versioning. 
This and other features can be modified in arguments.

In the example code, we perform two training rounds with slightly different models, allowing us to compare the results in the UI.

In [ ]:
mlflow.autolog() # https://mlflow.org/docs/latest/tracking/autolog.html#automatic-logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Let's compile two slightly different models to compare. You can either use the example code or insert your own.

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train / 255.
X_test = X_test / 255.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model_1 = Sequential(
    [
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ]
)
model_1.compile(optimizer='adagrad', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

model_2 = Sequential(
    [
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ]
)
model_2.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

models = {model_1: "3Layer_adagrad", 
          model_2: "3Layer_adam"}

for model, name in models.items():
    
    with mlflow.start_run(): # Trigger MLflow to start tracking the run: https://mlflow.org/docs/latest/python_api/mlflow.html?highlight=autolog#mlflow.start_run
        
        run_name = name # Assign an informative name to the run; otherwise, a random name will be generated.
        mlflow.set_tag("mlflow.runName", run_name) 
        print(f"Run name: {run_name}")

        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

        test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
        print('\nTest accuracy:', test_acc)
        
        mlflow.end_run() 

### User Interface

Next, we'll launch the MLflow application on Puhti and explore its user interface. You can find the MLflow icon under the Apps menu in Puhti. Ensure that the `pytorch/2.4` module is selected in the Settings section, as it includes a sufficiently recent version of MLflow. Additionally, make sure the file path points to the previously specified `mlruns` directory.

![mlflow_puhti.png](./pics/mlflow_puhti.png)

<small>Figure 1. Setup MLflow in Puhti</small>

After launching successfully, we can view the previous runs:

![run_1.png](./pics/runs_1.png)

<small>Figure 2. Front page of MLflow UI</small>

**Image caption:**

1. All experiments are listed here. With informative names and optional descriptions, user can organize different runs into easily manageable collections.
2. When selecting an experiment, all associated runs are displayed. User can sort and group these runs in various ways.

In the Charts view (Figure 3), users can compare the performance of different models using automatically generated graphs. These graphs can be downloaded in formats like CSV. 

![run_2.png](./pics/run_2.png)

<small>Figure 3. Charts- view of runs</small>

#### Conclusion

The Tracking Server and Models components together provide an easy way to centrally monitor and store machine learning experiments in a consistent manner. These tools are user-friendly and don’t require deep expertise to get started. However, MLflow also offers the flexibility for more detailed configuration to meet the needs of more demanding use cases.


---

## MLflow Model Registry

When you identify a model through experimentation that is ready for production, you can take advantage of the **[MLflow Model Registry](https://mlflow.org/docs/latest/model-registry.html#mlflow-model-registry)**. The registry offers a centralized platform for managing, validating, and deploying models. You can assign aliases such as "champion" or "challenger" to your models, making it easy to retrieve the appropriate model from the registry for inference.

The Model Registry can be managed through either the Models tab in the user interface or via the [API](https://mlflow.org/docs/latest/model-registry.html#adding-an-mlflow-model-to-the-model-registry).

![register_model.png](./pics/register_model.png)

<small>Figure 4. Register model in UI</small>

**Image caption:**

1. By opening the details of the desired run, you can register it from the top left corner.
2. Name the new model or select an existing one from the menu. If the model is already registered, a new version will be created.

<p style="color: red;">To complete following examples, register the `3Layer_adagrad` model, or the one with lower `validation_accuracy`.</p>

#### MLflow UI Models

Once the model is registered, you can view and manage it on the Models tab (Figure 5).

![model_reg1.png](./pics/model_reg1.png)

<small>Figure 5. Model Registry front page</small>

You can also manually add description, tags and alias (Figure 6).
<p style="color: red;">To complete following examples, add "champion" alias for the model registered before.</p>

![model_reg2.png](./pics/model_reg2.png)

<small>Figure 6. Details of a registered model</small>


### Registering a model via API

To register another trained model using the API, you need the run ID of the model you want to register. 

Since a previously registered model via the UI had a lower accuracy, we will now programmatically find and register the model with the highest `validation_accuracy`. 
Additionally, we will retrieve the model's name using API calls.

Finally, we assign the alias "challenger" to the model for easier identification. To accomplish this, we use the [MLflow Client](https://www.mlflow.org/docs/latest/python_api/mlflow.client.html?highlight=client#module-mlflow.client), which allows us to programmatically manage model aliases and streamline the process.


In [ ]:
from mlflow import MlflowClient
client = MlflowClient()

# https://mlflow.org/docs/latest/search-runs.html
runs = mlflow.search_runs([experiment.experiment_id]) 
run_id = runs.loc[runs['metrics.validation_accuracy'].idxmax(), 'run_id']

# https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.register_model
filter_string = "name LIKE 'mnist%'"
model_uri = f"runs:/{run_id}"
model_name = (mlflow.search_registered_models(filter_string=filter_string))[0].name
mv = mlflow.register_model(model_uri, model_name) 

# https://mlflow.org/docs/latest/python_api/mlflow.client.html#mlflow.client.MlflowClient.set_model_version_tag
client.set_registered_model_alias(mv.name, "challenger", mv.version)



When multiple versions of a model are available, they can be easily compared through the user interface (Figures 7 and 8).

![model_reg3.png](./pics/model_reg3.png)

<small>Figure 7. Model versions to compare</small>

![model_reg4.png](./pics/model_reg4.png)

<small>Figure 7. Comparing registered models</small>

### Deployment and Inference

As previously mentioned, the Model Registry allows for easy deployment of models using aliases.

In the following example code, we use the [**MLflow Models**](https://mlflow.org/docs/latest/models.html)- module to retrieve a previously registered model by its alias for deployment. The model is then utilized for inference through a lightweight [Gradio](https://www.gradio.app/) UI. Gradio is a user friendly and fast tool for demoing machine learning models.

When loading model versions, *model flavors* are used to ensure the model is loaded in the correct format. The [`pyfunc`](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#module-mlflow.pyfunc) flavor is the default and works with all models stored using MLflow. Additionally, MLflow supports various other [model flavors](https://mlflow.org/docs/latest/python_api/mlflow.models.html?highlight=flavors#module-mlflow.models), such as `sklearn`, `keras`, `pytorch`, and `xgboost`, which are specifically optimized for these libraries.


In [ ]:
# https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.load_model

alias = "champion"
model_name = "mnist_sequential"
model = mlflow.pyfunc.load_model(f"models:/{model_name}@{alias}") 
print(model)
print()

def classify(image):
    
    """
    This code processes an image by converting it to a NumPy array, normalizing pixel values (0-1), 
    and converting it to grayscale if it has 3 or 4 channels (RGB or RGBA). 
    It then resizes the image to 28x28 pixels and reshapes it into a (1, 28, 28) array.
    """
    
    composite_image = image['composite']
    composite_image = np.array(composite_image, dtype=np.float64)
    composite_image = composite_image / 255.0

    if composite_image.shape[-1] == 4:  
        composite_image = np.mean(composite_image[..., :3], axis=-1)
    elif composite_image.shape[-1] == 3:  
        composite_image = np.mean(composite_image, axis=-1)

    composite_image = resize(composite_image, (28, 28), anti_aliasing=True)
    composite_image = np.reshape(composite_image, (1, 28, 28))

    prediction = model.predict(composite_image).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

# https://www.gradio.app/docs

sketchpad = gr.Sketchpad()
label = gr.Label(num_top_classes=10)

interface = gr.Interface(fn=classify, inputs=sketchpad, outputs=label, live=False, allow_flagging="never")

interface.launch(share=True, debug=True)

---

## Projects (advanced)

[MLflow Projects](https://mlflow.org/docs/latest/projects.html) organizes code into **self-contained, reusable components** that can be run on different environments. A project in MLflow is essentially a **directory** or **Git repository** containing:

1. **Code (scripts)** for performing machine learning tasks.
2. **`MLproject` file**: A YAML configuration file that defines:
    - Project name
    - Dependencies (e.g., environment, for Puhti see https://docs.csc.fi/apps/python/#pre-installed-python-environments)
    - Entry points (main scripts/functions with parameters for tasks like training or evaluation)

The `MLproject` file might look like this:

```yaml
name: my_ml_project 
entry_points: 
	main: 
		parameters: learning_rate: {type: float, default: 0.01} 
		epochs: {type: int, default: 10} 
		command: | 
			module load ptensorflow
			python train.py --learning_rate {learning_rate} --epochs {epochs}


```
It would be utilized in a job script in the following manner:

```bash
#!/bin/bash
#SBATCH --job-name=mlflow_project
#SBATCH --output=mlflow_output_%j.log
#SBATCH --error=mlflow_error_%j.log
#SBATCH --time=00:01:00
#SBATCH --partition=test
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=1G
#SBATCH --account=project_20001234

module load python-data

mlflow run . -P learning_rate=0.01 -P epochs=10


```
For more information, see MLflow documentation.
